## 1) cloudstor access to files via webdav links

_`cloudstor` is a simple python tool to interface with the AARNET cloudstor (owncloud) deployment via webdav_

This is a simple webdav interface to the very specific implementation of the owncloud storage
solution that is available to all Australian researchers to store and share their data.
(See [https://www.aarnet.edu.au/.../cloudstor](https://www.aarnet.edu.au/network-and-services/cloud-services-applications/cloudstor) for details of this service). The github repository for this project is located at [https://github.com/underworldcode/cloudstor](https://github.com/underworldcode/cloudstor)


### Usage

The `cloudstor` class is based on the `webdav` class with some hard-coded paths that account for the
specific choices made in the AARNET implementation and is therefore largely a wrapper of convenience.

Cloudstor allows the sharing of files and directories via a url and these can have a password given 
to them (in the web browser interface) or left open. Directories have the option to be read / write.

### Directory links

A cloud store public link has the form "https://cloudstor.aarnet.edu.au/plus/s/s1z3yiL0j4sq3A9" and can 
be accessed directly from that link via a web browser. To download the information in the link, we 
need to know if it is a file or a directory. The following example is for a public link to a text file
with no password protection (it should be obvious what to do if you are given a password)

**Note**: you can also omit the "https://cloudstor.aarnet.edu.au/plus/s/" from your url as this is checked
and added if needed.

In [1]:
from cloudstor import cloudstor

public_ro = cloudstor(url="https://cloudstor.aarnet.edu.au/plus/s/F7yegRFEcsTH0QZ", password='ThisIsNotASecure1!!')

Cloudstore connection established


### Establish that this is a directory (not a file)

In [3]:
print(public_ro.remote_type)

directory


The remote "path" for a root directory object is either blank (`""`) or `"/"` 

In [9]:
print(public_ro.is_file("/"))
print(public_ro.is_dir(remote_path=""))

False
True


We can query the directory as we would a local one and retrieve a (python) list of the files and subdirectories and these can then be queried in turn by accessing the original `webdav` object.

In [13]:
ls_entries = public_ro.list()
print(ls_entries)
print(public_ro.is_file(ls_entries[0]))

['Readme.txt']
True


### Reading a remote file 

Use the same syntax as you would to open a file from the direct link but here with the remote path being parsed appropriately. 
**Note:** there is no magic here - the file is downloaded and removed when you have finished reading it. For large files, it is neither streamed nor cached. Consider downloading the file !

In [14]:
with public_ro.open("Readme.txt") as f:
    a = f.read()
    
print(a)

Test File (Password supplied)



### Downloading a remote file



In [15]:
public_ro.download_file("Readme.txt", "my_temporary_copy.txt")
!ls -al my_temporary_copy.txt
!rm  my_temporary_copy.txt

-rw-r--r--  1 lmoresi  staff  30 15 May 16:29 my_temporary_copy.txt


### Other directory operations

You can also `download` a directory with the expected results. 

In [22]:
public_ro.download("/", "tempdir")
!ls -al 
!echo "\ntempdir:" ; ls -al tempdir
!rm -rf tempdir

total 179776
drwxr-xr-x  27 lmoresi  staff       864 15 May 16:33 .
drwxr-xr-x  13 lmoresi  staff       416 14 May 15:17 ..
-rw-r--r--   1 lmoresi  staff      8196 14 May 15:15 .DS_Store
drwxr-xr-x   5 lmoresi  staff       160 15 May 16:23 .ipynb_checkpoints
-rw-r--r--   1 lmoresi  staff     12997 15 May 16:32 CloudstorDirectories.ipynb
-rw-r--r--   1 lmoresi  staff      5473 15 May 16:29 CloudstorFiles.ipynb
-rw-r--r--   1 lmoresi  staff      6993 15 May 16:22 CloudstorPrivateFiles.ipynb
-rw-r--r--   1 lmoresi  staff  11520134  8 May 20:17 Etopo1_2400x1200.tif
-rw-r--r--   1 lmoresi  staff  25920134  8 May 16:12 Etopo1_3600x1800.tif
-rw-r--r--   1 lmoresi  staff        33 15 May 11:34 Readme.txt
drwxr-xr-x  19 lmoresi  staff       608  2 May 16:59 Reference
drwxr-xr-x  42 lmoresi  staff      1344  2 May 16:59 Resources
-rw-r--r--   1 lmoresi  staff  12980154  9 May 11:05 events_4.5+by_year_1970-2019.npy
-rw-r--r--   1 lmoresi  staff        40 15 May 14:58 file.txt
-rw-r--r--   1 lmore

### Uploading and modifying remote files / directories

The sample resource is read-only, so the following operation fails:

In [26]:
try:
    public_ro.clean("Readme.txt")
except:
    print("Error: Forbidden operation")

Error: Forbidden operation


The following are available for links where read / write has been enabled (edit option ticked)

```python
    public_rw.download_directory(remote_directory, local_directory)  # immediate download of remote to local
    public_rw.sync(remote_directory, local_directory)                # 2 way sync of remote / local directory
    public_rw.download_async(remote_directory, local_directory)      # asynchronous download of remote to local
    public_rw.clean(remote_path)                                     # delete the remote path
    public_rw.move(remote_path_from, remote_path_to)                 # move the remote file / directory
    public_rw.copy(remote_path_from, remote_path_to)                 # move the remote file / directory

```

For obvious security / resource reasons, the upload and delete operations do not have examples provided here.